# 데이터 집계와 그룹연산

데이터 집계에서 다루어질 내용
> 하나 이상의 키(key)를 이용해서 pandas객체를 여러 조각으로 나누는 방법  
> 합계, 평균, 표준편자, 사용자 정의 함수 같은 그룹 요약 통계를 계산하는 방법  
> 정규화, 선형 회귀, 동급 또는 부분집합 선택 같은 집단 내 변형이나 다른 조작을 적용하는 방법  
> 피벗테이블과 교차 알람표를 구하는 방법  
> 변위치 분석과 다른 통계집단을 분석 수행하는 방법  

## GroupBy 메카닉

그룹 연산의 첫번째 단계에서는 Series, DataFrame 같은 pandas객체나 아니면 다른 객체에 들어 있는 데이터를 하나 이상의 키를 기준으로 분리
<img src ="https://media.vlpt.us/images/ssongplay/post/030290c1-825e-4ac9-8be5-b5f792f85a23/image.png" width=35%>

각 그룹의 색인은 다음과 같이 다양한 형태가 될 수 있으며 모두 같은 타입일 필요는 없다. 
- 그룹으로 묶을 축과 동일한 길이의 리스트나 배열
- DataFrame의 컬럼 이름을 지칭하는 값
- 그룹으로 묶을 값과 그룹 이름에 대응하는 사전이나 Series객체
- 축 색인 혹은 색인내의 개별 이름에 대해 실행되는 함수 

In [16]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randint(10,size=(5)),
                   'data2' : np.random.randint(10,size=(5))})
df

,key1,key2,data1,data2
0,a,one,8,5
1,a,two,2,6
2,b,one,2,3
3,b,two,2,0
4,a,one,6,1


In [17]:
df.groupby('key1').mean() # a의 값들을 data1과 data2의 평균 6 + 8 + 0 / 3을 구함

/var/folders/qz/7pf3mjnj3yzfv660r2vdk81w0000gn/T/ipykernel_39879/2945093410.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('key1').mean() # a의 값들을 data1과 data2의 평균 6 + 8 + 0 / 3을 구함


,data1,data2
key1,,
a,5.333333,4.0
b,2.000000,1.5


In [18]:
grouped = df['data1'].groupby(df['key1'])
grouped

데이터를 key1 으로 묶고 각 그룹에서 data1의 평균 구하기   
```groupBy```객체의 ```mean```메서드를 사용

In [19]:
grouped.mean()

key1
a    5.333333
b    2.000000
Name: data1, dtype: float64

여러개의 배열을 리스트로 넘기면 다음과 같은 결과로 두개의 색인으로 묶이고, 계층적인 색인을 가지는 ```Series```를 얻을 수 있음 

In [20]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     7.0
      two     2.0
b     one     2.0
      two     2.0
Name: data1, dtype: float64

In [21]:
means.unstack()

key2,one,two
key1,,
a,7.0,2.0
b,2.0,2.0


In [22]:
df

,key1,key2,data1,data2
0,a,one,8,5
1,a,two,2,6
2,b,one,2,3
3,b,two,2,0
4,a,one,6,1


In [23]:
df['data1']

0    8
1    2
2    2
3    2
4    6
Name: data1, dtype: int64

In [24]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2004, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean() # state로 먼저 묶고, 그다음 year, 그 후 mean

California  2005    2.0
            2006    2.0
Ohio        2004    8.0
            2005    2.0
            2006    6.0
Name: data1, dtype: float64

In [1]:
df.groupby('key1').mean()
df.groupby('key1').sum() # 더하기

NameError: name 'df' is not defined

In [26]:
df

,key1,key2,data1,data2
0,a,one,8,5
1,a,two,2,6
2,b,one,2,3
3,b,two,2,0
4,a,one,6,1


In [27]:

df.groupby(['key1', 'key2']).mean() # groupvy는 순서가 가장 중요

data1  data2
key1 key2              
a    one     7.0    3.0
     two     2.0    6.0
b    one     2.0    3.0
     two     2.0    0.0

In [28]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 그룹간 순회하기

```groupby``` 객체는 iteration을 지원하는데, 그룹 이름과 그에 따른 데이터 묶음을 튜플로 반환

In [29]:
df

,key1,key2,data1,data2
0,a,one,8,5
1,a,two,2,6
2,b,one,2,3
3,b,two,2,0
4,a,one,6,1


In [30]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)
    # 첫번째 키워드 두번째 키워드 

a
  key1 key2  data1  data2
0    a  one      8      5
1    a  two      2      6
4    a  one      6      1
b
  key1 key2  data1  data2
2    b  one      2      3
3    b  two      2      0


In [31]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2  data1  data2
0    a  one      8      5
4    a  one      6      1
('a', 'two')
  key1 key2  data1  data2
1    a  two      2      6
('b', 'one')
  key1 key2  data1  data2
2    b  one      2      3
('b', 'two')
  key1 key2  data1  data2
3    b  two      2      0


원하는 데이터만 고르기 위해서 그룹별 데이터를 사전형으로 쉽게 바꾸어 사용 가능 

In [32]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,2,3
3,b,two,2,0


```axis = 0``` 에 대해서 그룹을 만드는데 다른 축으로 그룹을 만드는 것도 가능 

아래의 예제는 df의 컬럼을 dtype에 따라 그룹으로 묶기 가능

In [18]:
df

,key1,key2,data1,data2
0,a,one,6,5
1,a,two,8,4
2,b,one,4,2
3,b,two,2,0
4,a,one,0,7


In [19]:
df.dtypes
grouped = df.groupby(df.dtypes, axis=1)

In [20]:
for dtype, group in grouped:
    print(dtype)
    print(group)

int64
   data1  data2
0      6      5
1      8      4
2      4      2
3      2      0
4      0      7
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 컬럼이나 컬럼의 일부만 선택하기

In [21]:
df

,key1,key2,data1,data2
0,a,one,6,5
1,a,two,8,4
2,b,one,4,2
3,b,two,2,0
4,a,one,0,7


In [22]:
df.groupby('key1')['data1']
#df.groupby('key1')[['data2']]

In [23]:
df['data1'].groupby(df['key1'])
a= df[['data2']].groupby(df['key1'])
for index, data in a:
    print(index)
    print(data)

a
   data2
0      5
1      4
4      7
b
   data2
2      2
3      0


아래의 예는 데이터에서 data2컬럼에 대해서만 평균을 구하고 결과를 ```DataFrame```으로 받고 싶다면 아래와 같이 작성

In [24]:
df['data2'].groupby([df['key1'], df['key2']]).mean()

key1  key2
a     one     6
      two     4
b     one     2
      two     0
Name: data2, dtype: int64

In [25]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2       
a    one       6
     two       4
b    one       2
     two       0

In [26]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped
s_grouped.mean()                                    # 위 2가지와 다 같은 결과를 보여준다. 

key1  key2
a     one     6
      two     4
b     one     2
      two     0
Name: data2, dtype: int64

### 사전과 Series에서 그룹핑하기

각 컬럼을 나타낼 그룹 목록이 있고, 그룹별로 컬럼의 값을 모두 더한다고 할 경우

In [27]:
people = pd.DataFrame(np.random.randint(10, size=(5,5)),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people


,a,b,c,d,e
Joe,9,6,3,1,0
Steve,6,8,8,5,1
Wes,2,5,4,1,2
Jim,2,2,3,0,2
Travis,7,9,4,8,6


In [28]:
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,9,6.0,3.0,1,0
Steve,6,8.0,8.0,5,1
Wes,2,NaN,NaN,1,2
Jim,2,2.0,3.0,0,2
Travis,7,9.0,4.0,8,6


In [33]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'} # 하나의 그룹처럼 생각해서 합을 구함

In [30]:
by_column = people.groupby(mapping, axis=1)
by_column.sum() 

,blue,red
Joe,4.0,15.0
Steve,13.0,15.0
Wes,1.0,4.0
Jim,3.0,6.0
Travis,12.0,22.0


In [31]:
map_series = pd.Series(mapping)
map_series


a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [32]:
people

,a,b,c,d,e
Joe,9,6.0,3.0,1,0
Steve,6,8.0,8.0,5,1
Wes,2,NaN,NaN,1,2
Jim,2,2.0,3.0,0,2
Travis,7,9.0,4.0,8,6


In [33]:
people.groupby(map_series, axis=1).count() # na는 치지 않음

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 함수로 그룹핑하기

In [34]:
people

,a,b,c,d,e
Joe,9,6.0,3.0,1,0
Steve,6,8.0,8.0,5,1
Wes,2,NaN,NaN,1,2
Jim,2,2.0,3.0,0,2
Travis,7,9.0,4.0,8,6


위의 people 데이터는  DataFrame은 사람의 이름을 색인값으로 사용.   
만약 사람의 이름의 길이 별로 그룹을 묶고 싶다면 길이가 담긴 배열을 만들어 넘기는 대신 ```len``` 함수 사용 가능. 

In [35]:
people.groupby(len).sum() # 이름의 길이로ㅓ 

,a,b,c,d,e
3,13,8.0,6.0,2,4
5,6,8.0,8.0,5,1
6,7,9.0,4.0,8,6


내부적으로는 모두 배열로 변환되므로 함수를 배열, 사전 또는 ```Series```와 함께 섞어 쓰더라도 전혀 문제가 되지 않음

In [36]:
people

,a,b,c,d,e
Joe,9,6.0,3.0,1,0
Steve,6,8.0,8.0,5,1
Wes,2,NaN,NaN,1,2
Jim,2,2.0,3.0,0,2
Travis,7,9.0,4.0,8,6


In [37]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min() # 길이가 3인 애들의 최소값을 찾아라 

a    b    c  d  e
3 one  2  6.0  3.0  1  0
  two  2  2.0  3.0  0  2
5 one  6  8.0  8.0  5  1
6 two  7  9.0  4.0  8  6

### 색인 단계로 그룹핑하기

계층적으로 색인된 데이터는 축 색인의 단계중 하나를 사용해서 편리하게 집계 가능

In [38]:

columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['city', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0     -0.334056 -1.541815  0.579270 -0.420507 -1.361043
1     -0.033068 -0.721204  1.071185 -2.018803 -0.721293
2      0.507892  1.141427  0.303120  0.450189 -0.583529
3     -0.393023 -0.155095 -1.641104 -0.767215  3.242798

```level``` 예약어를 사용해서 레벨 번호나 이름을 넘기면 가능

In [39]:
hier_df.groupby(level='city', axis=1).sum()

city,JP,US
0,-1.781550,-1.296601
1,-2.740095,0.316913
2,-0.133340,1.952438
3,2.475583,-2.189221


## 데이터 집계

In [40]:
df


,key1,key2,data1,data2
0,a,one,6,5
1,a,two,8,4
2,b,one,4,2
3,b,two,2,0
4,a,one,0,7


In [41]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.5)

key1
a    6.0
b    3.0
Name: data1, dtype: float64

자신만의 데이터 집계함수를 사용하려면 배열의 ```agg```메서드에 해당 함수를 넣으면 됨

In [42]:
def peak_to_peak(arr):
    return arr.max() - arr.min() # 최대에서 최소값 뺀것을 groupby에 적용하겠다. 
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,8,3
b,2,2


```describe```같은 메서드는 데이터를 집계하지 않는데도 잘 작동

In [43]:
grouped.describe()

data1                                              data2            \
     count      mean       std  min  25%  50%  75%  max count      mean   
key1                                                                      
a      3.0  4.666667  4.163332  0.0  3.0  6.0  7.0  8.0   3.0  5.333333   
b      2.0  3.000000  1.414214  2.0  2.5  3.0  3.5  4.0   2.0  1.000000   

                                         
           std  min  25%  50%  75%  max  
key1                                     
a     1.527525  4.0  4.5  5.0  6.0  7.0  
b     1.414214  0.0  0.5  1.0  1.5  2.0

### 컬럼에 여러가지 함수 적응하기

In [44]:
tips = pd.read_csv('./tips.csv')
tips.tail()

,total_bill,tip,smoker,day,time,size
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2
243,18.78,3.00,No,Thur,Dinner,2


In [45]:
tips = pd.read_csv('./tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


컬럼에 따라 다른 함수를 사용해서 집계를 수행 하거나 열개의 함수를 한번에 적용하기 원한다면 쉽고 간단하게 사용가능

In [46]:
grouped = tips.groupby(['day', 'smoker']) # 요일별, 담배를 피는별 그룹 바이

In [47]:
grouped_pct = grouped['tip_pct']  # 월화수는 없음
grouped_pct.agg('mean')           # 평균에 대한 팁 퍼센테이지

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

함수 목록이나 함수 이름을 넘기면 함수 이름을 컬럼으로 하는 ```DataFrame```을 얻을 수 있음

In [48]:
grouped_pct.agg(['mean', 'std', peak_to_peak]) # peak_to_peak : 최대에서 최소빼는 함수, list는 변수 그대로 가져감

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [49]:
grouped.agg({'tip' : np.max, 'size' : 'sum'}) # size 몇명이 밥을 먹었는지 # dict으로 묶으면 column이름으로 묶임

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [50]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

## Apply: 일반적인 분리-적용-병합

#### 상위 5개의 tip_pct 값을 고르기  
특정 칼럼에서 가장 큰 값을 갖는 행을 선택하는 함수 필요

In [51]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


smoker그룹에 대해서 이 함수를 적용하면 다음과 같은 결과

In [52]:
tips.groupby('smoker').apply(top) # tip데이터 엑셀 데이터를 비흡연자 흡연자로 나누고 행이이 5개씩 가져오라는 함수 

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

```apply``` 메서드를 넘길 함수가 추가적인 인자를 받는 다면 이 함수 이름 뒤에 붙여서 넘겨주면 가능 

In [53]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill') 

# 비흡연자 흡연자, 날짜별로 나누고 한명씩만 total-billl기준으로 갖고와라

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [54]:
result = tips.groupby('smoker')['tip_pct'].describe() 
# 흡연자 비흡연자의 기준으로 tip percent를 describe(개수, 평균, 표준편차std, min, 25, 50, 75, max)해라
result


,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [55]:
result.unstack('smoker') # 요일별로 흡연자 비흡연자로 나눈 데이터 

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [56]:
f = lambda x: x.describe()
grouped.apply(f) # world cloud

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### 그룹 색인 생략하기

In [57]:
tips.groupby('smoker', group_keys=False).apply(top) # 제일 처음에 있는 ?
# 흡연자 그룹바이 그룹바이 키=false로 하면 비흡연자 최대 5, 흡연자 최대 5의 옵션을 보여준다 안보여준다 차이

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [58]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

### 변위치 분석과 버킷분석

pandas의 ```cut```과 ```qcut```메서드를 사용하여 선택한 크기만큼 혹은 표분 변위치에 따라 데이터를 나눌 수 있음  
cut을 이용해서 등간격 구간으로 나누기


In [59]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0     (-1.354, 0.137]
1      (0.137, 1.629]
2     (-1.354, 0.137]
3     (-1.354, 0.137]
4      (0.137, 1.629]
5     (-1.354, 0.137]
6     (-1.354, 0.137]
7    (-2.852, -1.354]
8     (-1.354, 0.137]
9     (-1.354, 0.137]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.852, -1.354] < (-1.354, 0.137] < (0.137, 1.629] < (1.629, 3.12]]

```cut```에서 반환된 categorical객체는 바로 ```groupby```로 넘기기 가능 

In [60]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles) # 4분의 1
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.852, -1.354]",-2.670837,2.165191,96.0,-0.056698
"(-1.354, 0.137]",-2.845968,2.621140,466.0,0.052759
"(0.137, 1.629]",-2.730936,4.081882,375.0,0.062694
"(1.629, 3.12]",-2.021397,2.336582,63.0,0.114809


표본 변위치에 기반하여 크기각 같은 버킷을 계산하기 위해서는 ```qcut```을 사용

In [61]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False) # qcut 개수 새가지고 10개로 나눔
grouped = frame.data2.groupby(grouping) # cut = 범위를 4등분
grouped.apply(get_stats).unstack() 

,min,max,count,mean
data1,,,,
0,-2.670837,2.165191,100.0,-0.080593
1,-2.365677,2.408406,100.0,-0.008987
2,-2.149513,2.390371,100.0,0.184373
3,-2.845968,2.227815,100.0,0.006810
4,-2.144155,2.621140,100.0,0.112449
5,-2.372900,2.123329,100.0,-0.025440
6,-1.855441,2.650948,100.0,0.106679
7,-2.730936,2.370542,100.0,0.003953
8,-2.238333,2.189113,100.0,-0.031431


### Example: 그룹에 따른 값으로 결측치 채우기

누락된 데이터를 정리할때 어떤 경우에는 dropna를 사용해서 데이터를 살펴보고 걸러내기가능  
어떤 경우에는 누락된 값을 고정된 값이나 데이터로부터 도출된 어떤 값으로 채우고 싶을때에는 fillna메서드를 사용
누락된 값을 평균값으로 대체 


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s


0         NaN
1   -0.690765
2         NaN
3    0.510039
4         NaN
5    0.713117
dtype: float64

In [4]:
s.fillna(s.mean()) # 누락된 값을 평균값으로 대체

0    0.177464
1   -0.690765
2    0.177464
3    0.510039
4    0.177464
5    0.713117
dtype: float64

In [5]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4 # 앞의 키값은 east, 뒤 키 값은 west
data = pd.Series(np.random.randint(10,size=(8)), index=states)
data

Ohio          3
New York      3
Vermont       6
Florida       6
Oregon        3
Nevada        2
California    0
Idaho         6
dtype: int64

In [6]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          3.0
New York      3.0
Vermont       NaN
Florida       6.0
Oregon        3.0
Nevada        NaN
California    0.0
Idaho         NaN
dtype: float64

In [7]:
data.groupby(group_key).mean()

East    4.0
West    1.5
dtype: float64

In [8]:
fill_mean = lambda g: g.fillna(g.mean()) # 데이터를 갖고오는데 평균값으로 채워라.
data                                     # 그룹별로 평균값을 채워라 가능하겠죠?

Ohio          3.0
New York      3.0
Vermont       NaN
Florida       6.0
Oregon        3.0
Nevada        NaN
California    0.0
Idaho         NaN
dtype: float64

In [68]:
data.groupby(group_key).apply(fill_mean)

Ohio          1.000000
New York      5.000000
Vermont       4.333333
Florida       7.000000
Oregon        9.000000
Nevada        8.500000
California    8.000000
Idaho         8.500000
dtype: float64

In [69]:
fill_values = {'East': 0.5, 'West': -1} # east로 오면 0.5, west는 -1로 채우겠다는 의미
fill_func = lambda g: g.fillna(fill_values[g.name]) # 
data.groupby(group_key).apply(fill_func)

Ohio          1.0
New York      5.0
Vermont       0.5
Florida       7.0
Oregon        9.0
Nevada       -1.0
California    8.0
Idaho        -1.0
dtype: float64

### Example: 그룹의 가중 평균과 상관관계

In [70]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,1.476614,0.713777
1,a,0.406252,0.627438
2,a,0.544388,0.446380
3,a,-0.385313,0.937445
4,b,0.911530,0.972048
5,b,-0.724421,0.216456
6,b,0.788731,0.874631
7,b,-0.484583,0.391654


In [71]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.436935
b    0.500778
dtype: float64

야후의 파이낸스에서 가져온 몇몇 주식과 s&p 500 지수(종목코드 SPX)의 종가 데이터를 살펴보자

In [72]:
close_px = pd.read_csv('./stock_px_2.csv', parse_dates=True,
                       index_col=0)
close_px.info()
close_px[-4:]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


퍼센트의 변화율로 일일 수익률을 계산하여 연간 SPX 지수와의 상관 관계를 알아보기

In [73]:
spx_corr = lambda x: x.corrwith(x['SPX'])

```pct_change```함수를 이용해서 close_px의 페선트 변화율을 계산

In [74]:
rets = close_px.pct_change().dropna()

datetime에서 연도 속성만 반환하는 한줄짜리 함수를 이용하여 연도별 퍼센트 변화율

In [75]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year) # 상관관계 분석
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


아래는 애플과 마이크로 소프트의 주가의 연간 상관관계

In [76]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

## 피벗테이블과 교차 일람표

피벗테이블은 앞에서 설명한 groupby 기능을 사용해서 측정 색인 활용한 재형성 연산가능


DataFrame에는 pivot_table 메서드가능


In [77]:
tips.pivot_table(index=['day', 'smoker']) # 인덱스 지정하고 흡연 2차로 분류 사이즈 팁 퍼센테이지 총 값 
# pivot_table을 쓰던지 groupby를 쓰던지 

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [78]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker') # 퍼센테이지랑 몇명인지

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [79]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [80]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [81]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### Cross-Tabulations: Crosstab

In [82]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [83]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [84]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [85]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244